In [1]:
from geopy.geocoders import Nominatim
import geopy
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.extra.rate_limiter import RateLimiter
from shapely.geometry import Polygon
import time

In [4]:
df = pd.read_csv("data/processed/processed_municip_kWh.csv", encoding="utf-8", index_col=0)
df.head() 

,Province,Municipality,Month,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree)
0,Alberta,Acadia Valley,January,3.78,3.57,3.83,3.11,4.31,1.23
1,Alberta,Acadia Valley,February,4.69,4.78,4.96,4.35,5.87,2.23
2,Alberta,Acadia Valley,March,4.79,5.58,5.49,5.37,7.20,3.63
3,Alberta,Acadia Valley,April,4.09,5.80,5.34,5.96,8.43,4.96
4,Alberta,Acadia Valley,May,3.37,5.59,4.88,6.06,9.06,5.84


In [5]:
df_annual = df[df['Month'].str.contains("Annual", na=False)].copy()
df_annual.loc[:, 'address'] = df_annual.loc[:, 'Municipality'] + ', ' + df_annual.loc[:, 'Province']
df_annual.reset_index(inplace=True, drop=True)
df_annual.head() 

,Province,Municipality,Month,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree),address
0,Alberta,Acadia Valley,Annual,3.85,4.95,4.71,4.94,7.09,3.73,"Acadia Valley, Alberta"
1,Alberta,Acme,Annual,3.72,4.76,4.53,4.76,6.77,3.62,"Acme, Alberta"
2,Alberta,Airdrie,Annual,3.66,4.70,4.47,4.70,6.67,3.61,"Airdrie, Alberta"
3,Alberta,Alberta Beach,Annual,3.52,4.44,4.21,4.46,6.35,3.42,"Alberta Beach, Alberta"
4,Alberta,Alder Flats,Annual,3.47,4.41,4.18,4.42,6.28,3.44,"Alder Flats, Alberta"


In [6]:
address = df_annual.iloc[0]["address"]
address

'Acadia Valley, Alberta'

In [7]:
len(df_annual['address'].unique())

3506

In [9]:
# # try running this block first to see if can create latitude and longtitude of one single location 
# loc = Nominatim(user_agent="Geopy Library")
# getLoc = loc.geocode(address)

# print(getLoc.address)
# print("Latitude = ", getLoc.latitude, "\n")
# print("Longitude = ", getLoc.longitude)

Acadia Valley, Division No. 4, Alberta, Canada
Latitude =  51.1586763 

Longitude =  -110.210332


In [10]:
# geolocator = geopy.Nominatim(user_agent="Geopy Library")
# lat = []
# lon = []

# for i, address in enumerate(df_annual.address):
#     try:
#         getLoc = geolocator.geocode(address)
#         lat.append(getLoc.latitude)
#         lon.append(getLoc.longitude)
#     except:
#         lat.append(pd.NA)
#         lon.append(pd.NA)

#     if i % 100 == 0:
#         print(f'Index {i} completed')
#         print(lat)
#         print(lon)


Index 0 completed
[51.1586763]
[-110.210332]


In [12]:
# len(lat), len(lon)

(50, 50)

In [2]:
coordinate_df = pd.read_csv('data\processed\processed_municip_kWh_with_coordinates.csv')
coordinate_df.head(7)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\fohy2\AppData\Local\Temp\ipykernel_59988\1671886408.py:1: SyntaxWarning: invalid escape sequence '\p'
  coordinate_df = pd.read_csv('data\processed\processed_municip_kWh_with_coordinates.csv')


,Province,Municipality,Month,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree),address,latitude,longitude
0,Alberta,Acadia Valley,Annual,3.85,4.95,4.71,4.94,7.09,3.73,"Acadia Valley, Alberta",51.158676,-110.210332
1,Alberta,Acme,Annual,3.72,4.76,4.53,4.76,6.77,3.62,"Acme, Alberta",51.497782,-113.513721
2,Alberta,Airdrie,Annual,3.66,4.70,4.47,4.70,6.67,3.61,"Airdrie, Alberta",51.285970,-114.010620
3,Alberta,Alberta Beach,Annual,3.52,4.44,4.21,4.46,6.35,3.42,"Alberta Beach, Alberta",53.674589,-114.357227
4,Alberta,Alder Flats,Annual,3.47,4.41,4.18,4.42,6.28,3.44,"Alder Flats, Alberta",52.931756,-114.959071
5,Alberta,Alix,Annual,3.69,4.67,4.44,4.67,6.65,3.54,"Alix, Alberta",52.394015,-113.186446
6,Alberta,Alliance,Annual,3.74,4.74,4.50,4.74,6.78,3.57,"Alliance, Alberta",52.433555,-111.783236


In [3]:
import requests

def get_polygon_coordinates_osm(lat, lon):

    # lat, lon = 51.497782, -113.513721
    
    # Reverse Geocoding: Get polygon coordinates for the latitude and longitude
    reverse_geocode_url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json&polygon_geojson=1"
    reverse_geocode_response = requests.get(reverse_geocode_url)
    if reverse_geocode_response.status_code != 200:
        return None
    reverse_geocode_data = reverse_geocode_response.json()
    if 'geojson' in reverse_geocode_data and 'coordinates' in reverse_geocode_data['geojson']:
        return reverse_geocode_data['geojson']['coordinates']
    return None

polygon_coordinates  = get_polygon_coordinates_osm(52.433555, -111.783236)
polygon_coordinates


[[-111.7817401, 52.433109], [-111.7836416, 52.4341663]]

In [4]:
polygon_list = []
for i, (lat, lon) in enumerate(zip(coordinate_df['latitude'], coordinate_df['longitude'])):
    try:
        polygon_coordinates = get_polygon_coordinates_osm(lat, lon)
        try:
            polygon_list.append(Polygon(polygon_coordinates))
        except:
            polygon_list.append(Polygon(polygon_coordinates[0]))
    except:
        polygon_list.append(np.nan)
    
    if i % 100 == 0:
        print(f'Index {i} completed')
        print(polygon_list[-100:])
polygon_list

Index 0 completed
[<POLYGON ((-110.214 51.159, -110.211 51.159, -110.21 51.159, -110.209 51.159...>]
Index 100 completed
[<POLYGON ((-113.514 51.499, -113.514 51.498, -113.514 51.497, -113.514 51.49...>, <POLYGON ((-114.011 51.286, -114.011 51.286, -114.011 51.286, -114.011 51.28...>, <POLYGON ((-114.357 53.675, -114.357 53.674, -114.357 53.674, -114.357 53.67...>, <POLYGON ((-114.968 52.931, -114.967 52.931, -114.966 52.931, -114.964 52.93...>, nan, nan, <POLYGON ((-110.163 51.918, -110.161 51.918, -110.161 51.918, -110.161 51.91...>, <POLYGON ((-111.061 52.565, -111.059 52.564, -111.058 52.565, -111.06 52.566...>, <POLYGON ((-112.331 53.879, -112.331 53.879, -112.331 53.879, -112.331 53.87...>, <POLYGON ((-111.055 56.462, -111.054 56.462, -111.052 56.461, -111.051 56.46...>, nan, nan, <POLYGON ((-111.569 54.13, -111.568 54.13, -111.566 54.13, -111.569 54.13))>, <POLYGON ((-118.682 58.699, -118.683 58.699, -118.683 58.699, -118.683 58.69...>, <POLYGON ((-112.93 54.761, -112.928 54.774

[<POLYGON ((-110.214 51.159, -110.211 51.159, -110.21 51.159, -110.209 51.159...>,
 <POLYGON ((-113.514 51.499, -113.514 51.498, -113.514 51.497, -113.514 51.49...>,
 <POLYGON ((-114.011 51.286, -114.011 51.286, -114.011 51.286, -114.011 51.28...>,
 <POLYGON ((-114.357 53.675, -114.357 53.674, -114.357 53.674, -114.357 53.67...>,
 <POLYGON ((-114.968 52.931, -114.967 52.931, -114.966 52.931, -114.964 52.93...>,
 nan,
 nan,
 <POLYGON ((-110.163 51.918, -110.161 51.918, -110.161 51.918, -110.161 51.91...>,
 <POLYGON ((-111.061 52.565, -111.059 52.564, -111.058 52.565, -111.06 52.566...>,
 <POLYGON ((-112.331 53.879, -112.331 53.879, -112.331 53.879, -112.331 53.87...>,
 <POLYGON ((-111.055 56.462, -111.054 56.462, -111.052 56.461, -111.051 56.46...>,
 nan,
 nan,
 <POLYGON ((-111.569 54.13, -111.568 54.13, -111.566 54.13, -111.569 54.13))>,
 <POLYGON ((-118.682 58.699, -118.683 58.699, -118.683 58.699, -118.683 58.69...>,
 <POLYGON ((-112.93 54.761, -112.928 54.774, -112.927 54.774, -112.

In [6]:
coordinate_df['polygons'] = polygon_list
coordinate_df.head()

,Province,Municipality,Month,South-facing with vertical (90 degrees) tilt,South-facing with latitude tilt,South-facing with tilt=latitude+15 degrees,South-facing with tilt=latitude-15 degrees,2-axis tracking,Horizontal (0 degree),address,latitude,longitude,polygons
0,Alberta,Acadia Valley,Annual,3.85,4.95,4.71,4.94,7.09,3.73,"Acadia Valley, Alberta",51.158676,-110.210332,"POLYGON ((-110.2141432 51.1587175, -110.211333..."
1,Alberta,Acme,Annual,3.72,4.76,4.53,4.76,6.77,3.62,"Acme, Alberta",51.497782,-113.513721,"POLYGON ((-113.5143844 51.4985257, -113.514325..."
2,Alberta,Airdrie,Annual,3.66,4.70,4.47,4.70,6.67,3.61,"Airdrie, Alberta",51.285970,-114.010620,"POLYGON ((-114.0107674 51.2860953, -114.010766..."
3,Alberta,Alberta Beach,Annual,3.52,4.44,4.21,4.46,6.35,3.42,"Alberta Beach, Alberta",53.674589,-114.357227,"POLYGON ((-114.3570561 53.6745442, -114.356754..."
4,Alberta,Alder Flats,Annual,3.47,4.41,4.18,4.42,6.28,3.44,"Alder Flats, Alberta",52.931756,-114.959071,"POLYGON ((-114.9682284 52.9313586, -114.967447..."


In [7]:
coordinate_df.to_csv('data\processed\processed_municip_kWh_with_polygons.csv', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\fohy2\AppData\Local\Temp\ipykernel_59988\3073207043.py:1: SyntaxWarning: invalid escape sequence '\p'
  coordinate_df.to_csv('data\processed\processed_municip_kWh_with_polygons.csv', index=False)


In [9]:
coordinate_df['polygons'].isna().sum()

1341

In [24]:
# then run this 

if 'latitude' not in df_annual.columns:
    df_annual['latitude'] = pd.NA
if 'longitude' not in df_annual.columns:
    df_annual['longitude'] = pd.NA

# Set up the geolocator with rate limiting
geolocator = geopy.Nominatim(user_agent="Geopy Library")
rate_limiter = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # adjust min_delay_seconds as needed

for index, row in df_annual.iterrows():
    
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):  # Check if geocoding is needed
        try:
            location = rate_limiter(row['address'])
            if location:
                df_annual.at[index, 'latitude'] = location.latitude
                df_annual.at[index, 'longitude'] = location.longitude
        except geopy.exc.GeocoderServiceError as e:
            print(f"Error geocoding {row['address']}: {e}")

print(df_annual.head())  # Show the first few rows to check

KeyboardInterrupt: 